<a href="https://colab.research.google.com/github/Jagaya/FCC-Machine-Learning/blob/main/predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, StandardScaler

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset_numerical = dataset.copy()

In [ ]:
dataset.region.value_counts()

In [ ]:
dataset_numerical["sex"] = dataset["sex"].map({"female":0, "male": 1}).astype(int)
dataset_numerical["smoker"] = dataset["smoker"].map({"no":0, "yes":1})

for reg in ["southeast", "northwest", "southwest", "northeast"]:
  dataset_numerical[reg] = dataset_numerical.region == reg

# region_type = tf.feature_column.categorical_column_with_vocabulary_list(
#       'region', ["southeast", "southwest", "northwest", "northeast"])

# region_type_one_hot = tf.feature_column.indicator_column(region_type)

In [ ]:
# dataset_numerical["region"]

In [ ]:
dataset_numerical.columns

In [ ]:
# dataset_numerical.drop(["region"], inplace=True, axis=1)

In [ ]:
dataset_numerical = dataset_numerical[['age', 'sex', 'bmi', 'children', 'smoker', 'southeast',
       'northwest', 'southwest', 'northeast', 'expenses']]

In [ ]:
dataset_numerical.fillna(0).head()

In [ ]:
train_data, test_data = train_test_split(dataset_numerical, test_size = 0.2)

In [ ]:
print(len(train_data), 'train examples')
# print(len(val), 'validation examples')
print(len(test_data), 'test examples')

In [ ]:
to_scale = ["age", "bmi"]
scaler = StandardScaler()

scaler.fit(train_data.loc[:, to_scale])
train_data.loc[:, to_scale] = scaler.transform(train_data.loc[:, to_scale])
test_data.loc[:, to_scale] = scaler.transform(test_data.loc[:, to_scale])

In [ ]:
train_data.head()

In [ ]:
train_labels = train_data["expenses"].astype(np.float)
train_dataset = train_data.drop(["expenses"], axis=1).astype(np.float)

In [ ]:
train_dataset

In [ ]:
model = tf.keras.Sequential([
              keras.layers.Dense(64, activation="relu"),
              keras.layers.Dense(64, activation ="relu"),
              keras.layers.Dense(1, activation="relu")
])

In [ ]:
model.compile(
    optimizer = "adam",
    loss = "mean_squared_error",
    metrics = ["mae", "mse"]
)

In [ ]:
## doesn't work if there is no input-layer pre-defined
# model.summary()

In [ ]:
history = model.fit(train_dataset, train_labels, epochs = 100, verbose=1)

In [ ]:
result = model.evaluate(test_dataset, test_labels, verbose=2)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
# loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
